# 컴퓨터 비전 딥러닝
## 신경망 첫걸음

### MNIST 데이터 가져오기



In [ ]:
import torch
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt


In [ ]:
transformation = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307), (0.3081))])

train_dataset = datasets.MNIST('data/', train=True, transform=transformation, download=True)

test_dataset = datasets.MNIST('data/', train=False, transform=transformation, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
def plot_img(image):
    image = image.numpy()[0]
    mean = 0.1307
    std = 0.3081
    image = ((mean * image) + std)
    plt.imshow(image, cmap='gray')

In [ ]:
sample_data = next(iter(train_loader))
plot_img(sample_data[0][1])
plot_img(sample_data[0][2])

### CNN 모델 구축

In [ ]:
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

### **conv2d**

conv2는 MNIST 이미지에 컨볼루션 필터를 적용한다.

컨볼루션은 스트라이드(stride 기본값이 1인 매개변수)의 값을 따라 옆으로 이동하며 필터를 입력에 적용한다. 여기서 필터는 가중치의 집합을 의미한다. 

패딩(padding)이라는 매개변수도 있다. 패딩이란 데이터가 부족해서 스트라이드를 할 수 없을 때 양 끝에 0을 추가해서 스트라이드가 가능하게 만드는 것이다. 

마지막으로 kernel_size는 사용되는 커널의 크기를 결정한다. 일반적으로 1,3,5,7이 사용된다. 초기엔 일반적으로 7이나 9의 필터를 사용한다.

참고 ( http://setosa.io/ev/image-kernels/ )

CNN은 컨볼루션 레이어와 선형 레이어로 구분된다. 컨볼루션 레이어에서 신경망이 학습하는 학습 파라미터는 커널의 파라미터다. 각 컨볼루션 레이어의 선형 레이어에서 이미지 분류하기 좋은 형태로 입력 데이터를 필터로 변경시키는 작업을 수행한다. CNN의 학습 과정에서 컨볼루션 레이어의 목표는 이미지를 가장 효과적인 변환 커널을 학습하는 것이다. 이렇게 CNN 컨볼루션 레이어의 학습 대상은 커널 파라미터가 된다..?

### **풀링**
컨볼루션 다음에 풀링을 추가하는 것은 일반적이다.

풀링은 크게 2가지 기능을 제공한다. 하나는 처리할 데이터를 줄이는 것, 또다른 하나는 알고리즘의 이미지 위치의 작은 변화에 집중하지 않도록 만드는 것이다. 

풀링은 약간 컨볼루션과 비슷하게 커널크기, 스트라이드는 동일하게 갖고 있으나 가중치가 없다.

MaxPool2d의 경우에는 커널 크기안에서 최댓값만 남기고 줄이게 된다.
AveragePooling도 있다. 

### **비선형 활성화 레이어 ReLU**

일반적으로 컨볼루션을 수행한 후 맥스 풀링을 적용한다. 그 다음 비선형 레이어를 연결하는 형태를 주로 사용한다.

### **뷰**

이미지 분류 문제를 다루는 네트워크 대부분 마지막에 전연결 또는 선형 레이어를 배치하는 것이 일반적이다. 뷰는 컨볼루션의 출력으로 나온 2차원 텐서를 선형 레이어에 연결하기 위해 1차원 텐서로 바꾸어주는 역할을 한다.

뷰 함수에 전달하는 첫번째 인수는 파이토치가 첫번째 차원에서 데이터를 병합하지 않도록 지시한다. 첫번째 인자가 -1이 된다면 그 다음 인자의 크기로 첫번째 인자를 계산하겠다는 의미이다.(아직 잘은 모르겠다...)

2차원 텐서를 1차원으로 변환한 후 선형 레이어에 전달하여 그 출력을 다시 비선형 활성화 레이어에 전달한다. 

### **모델학습**



In [ ]:
def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0

    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = Variable(data, volatile), Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        running_loss += F.nll_loss(output, target, size_average=False).data[0]
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == "training":
            loss.backward()
            optimizer.step()
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100 * running_correct / len(data_loader.dataset)

    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss, accuracy

## 전이학습(CGG16)

In [ ]:
from torchvision import models 
vgg = models.vgg16(pretrained= True)


In [ ]:
vgg

vgg는 크게 features, classifier로 구성되어있다.
vgg의 가중치는 이미 훈련돼 있기 때문에 레이어의 가중치 또는 매개변수를 고정한다. 이를 통해 옵티마이저가 가중치를 업데이트 하지 못한다.

In [ ]:
for param in vgg.features.parameters():
    param.requires_grad = False

In [ ]:
vgg.classifier[6].out_features = 2

In [ ]:
optimizer = torch.optim.SGD(vgg.classifier.parameters(), lr=0.0001, momentum=0.5)

컨볼루션 레이어의 parameter를 고정하고 학습을 한다면 항상 컨볼루션의 학습결과는 변하지 않을 것이다. 그렇다면 에폭을 반복할 때마다 굳이 컨볼루션 학습을 시킬 이유가 없다. 이럴 때 컨볼루션만 따로 결과를 저장하여 개별 실행할 수도 있다.

In [ ]:
features = vgg.features

train_data_loader = torch.utils.data.DataLoader(train, batch_size=32, num_workers=3, shuffle=False)
valid_data_loader = torch.utils.data.DataLoader(valid, batch_size=32, num_workers=3, shuffle=False)
def preconvfeat(dataset, model):
    conv_features = []
    labels_list = []
    for data in dataset:
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        output = model(inputs)
        conv_features.extend(output.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])

    return (conv_featrues, labels_list)

conv_feat_train, labels_train = preconvfeat(train_data_loader, features)
conv_feat_val, labels_val = preconvfeat(valid_data_loader, features)

학습 데이터셋과 검증 데이터셋에 대한 컨볼루션 연산 결과를 저장한 후 Dataset 클래스를 생성해 새로운 클래스를 만들고 이 클래스를 사용해 DataLoader 객체를 생성한다.
DataLoader 객체를 이용하면 학습 과정을 간단하게 만들 수 있다.
컨볼루션 특성을 위한 데이터 셋 클래스를 정의하고 DataLoader 객체를 생성하는 코드는 다음과 같다. 학습 및 검증 데이터셋에 대한 컨볼루션 기능이 있으면 파이토치 데이터셋 및 dataLoader 클래스를 만들어 교육 과정을 가노화 한다. 다음 코드는 컨볼루션 기능을 위한 데이터셋 및 데이터 로더를 만든다.

In [ ]:
from torch.utils.data import Dataset

class My_dataset(Dataset):
    def __init__(self, feat, labels):
        self.conv_feat = feat
        self.labels = labels
    
    def __len__(self):
        return len(self.conv_feat)
    
    def __getitem(self, idx):
        return self.conv_feat[idx], self.labels[idx]
    
train_feat_loader = My_dataset(conv_feat_train, labels_train)
val_feat_dataset = My_dataset(conv_feat_val, labels_val)

train_feat_loader = DataLoader(train_feat_dataset, batch_size=64, shuffle=True)
val_feat_loader = DataLoader(val_feat_dataset, batch_size=64, shuffle=True)